In [18]:
import pandas as pd
import os
import ast
import numpy as np
import tensorflow as tf
import einops

from keras.layers import Input, Dense, concatenate, Embedding, ReLU, Dropout, Bidirectional, GRU, Add, Concatenate
from keras import Model

In [19]:
UNITS = 256

In [20]:
os.getcwd()

'c:\\Users\\salha\\studium\\NLP\\projektabeit_nlp_2\\analyse_phase_01'

In [21]:
text_path = os.path.join('data', 'choc_recipes.txt')
# Using readlines()
file1 = open(text_path, 'r')
Lines = file1.readlines()

names = []
ingredients = []
steps = []

count = 0
# Strips the newline character
for line in Lines:
    pieces = line.split('\t')
    names.append(pieces[0])
    ingredients.append(pieces[1])
    steps.append(pieces[2])

In [22]:
print(f'Name count: {len(names)}')
print(f'Ingredient count: {len(ingredients)}')
print(f'Step count: {len(steps)}')

Name count: 7404
Ingredient count: 7404
Step count: 7404


In [23]:
class ShapeChecker():
  def __init__(self):
    # Keep a cache of every axis-name seen
    self.shapes = {}

  def __call__(self, tensor, names, broadcast=False):
    if not tf.executing_eagerly():
      return

    parsed = einops.parse_shape(tensor, names)

    for name, new_dim in parsed.items():
      old_dim = self.shapes.get(name, None)

      if (broadcast and new_dim == 1):
        continue

      if old_dim is None:
        # If the axis name is new, add its length to the cache.
        self.shapes[name] = new_dim
        continue

      if new_dim != old_dim:
        raise ValueError(f"Shape mismatch for dimension: '{name}'\n"
                         f"    found: {new_dim}\n"
                         f"    expected: {old_dim}\n")

In [24]:
ingredients_list_list = []
for i_string in ingredients:
    i = i_string.replace(', ', '*')
    i = i.replace(' ', '_')
    i = i.replace('*', ', ')

    ingredients_list_list.append(i)

In [25]:
ingredients = ingredients_list_list

In [26]:


name_input = np.array(names)
ingredient_input = np.array(ingredients)
step_output = np.array(steps)

ingredient_dataset = tf.data.Dataset.from_tensor_slices(ingredient_input)
name_dataset =  tf.data.Dataset.from_tensor_slices(name_input)
step_dataset =  tf.data.Dataset.from_tensor_slices(step_output)
#dataset_12 = tf.data.Dataset.from_tensor_slices((name_input, ingredient_input))
#dataset_label = tf.data.Dataset.from_tensor_slices(step_output)

#dataset = tf.data.Dataset.zip((dataset_12, dataset_label)).batch(64)
#model.fit(dataset, epochs=10, steps_per_epoch=4)

In [27]:
max_features = 10000
max_len = 400


step_vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

#train_raw.map(lambda context, target: context)
step_vectorize_layer.adapt(step_dataset)

In [28]:
max_features = 2000
max_len = 4


ingredient_vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

#train_raw.map(lambda context, target: context)

ingredient_vectorize_layer.adapt(ingredients)

In [29]:
max_features = 5000
max_len = 4


name_vectorize_layer = tf.keras.layers.TextVectorization(
 max_tokens=max_features,
 output_mode='int',
 output_sequence_length=max_len)

#train_raw.map(lambda context, target: context)
name_vectorize_layer.adapt(name_dataset)

In [30]:
name_vectorize_layer.get_vocabulary()

['',
 '[UNK]',
 'chocolate',
 'cake',
 'cookies',
 'chip',
 's',
 'brownies',
 'fudge',
 'butter',
 'peanut',
 'bars',
 'cream',
 'pie',
 'with',
 'and',
 'white',
 'hot',
 'pudding',
 'brownie',
 'muffins',
 'easy',
 'no',
 'cheesecake',
 'oatmeal',
 'banana',
 'double',
 'mocha',
 'cocoa',
 'cookie',
 'cupcakes',
 'caramel',
 'bread',
 'the',
 'free',
 'cherry',
 'low',
 'bake',
 'orange',
 'mousse',
 'frosting',
 'coffee',
 'ice',
 'fat',
 'squares',
 'sauce',
 'coconut',
 'almond',
 'best',
 'dark',
 'mix',
 'dessert',
 'raspberry',
 'mint',
 'toffee',
 'chewy',
 'cheese',
 'pecan',
 'vegan',
 'for',
 'black',
 'a',
 'rich',
 'nut',
 'truffles',
 'balls',
 'fudgy',
 'candy',
 'layer',
 'mexican',
 'triple',
 'or',
 'biscotti',
 'walnut',
 'of',
 'chunk',
 'bar',
 'cranberry',
 'cinnamon',
 'in',
 'decadent',
 'bundt',
 'kahlua',
 'milk',
 'creamy',
 'cakes',
 'marshmallow',
 'espresso',
 'sugar',
 'oreo',
 'torte',
 'pumpkin',
 'gluten',
 'strawberry',
 'one',
 'icing',
 'mini',
 '

In [56]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, name_processor, ingredient_processor, units):
    super(Encoder, self).__init__()
    
    self.name_processor = name_processor
    self.ingredient_processor = ingredient_processor

    self.name_vocab_size = name_processor.vocabulary_size()
    self.ingredient_vocab_size = ingredient_processor.vocabulary_size()
    
    self.units = units

    # The embedding layer converts tokens to vectors
    self.name_embedding = tf.keras.layers.Embedding(self.name_vocab_size, units, mask_zero=True)
    self.ingredient_embedding = tf.keras.layers.Embedding(self.ingredient_vocab_size, units, mask_zero=True)

    self.concat = Concatenate()

    self.embedding = tf.keras.layers.Embedding((self.name_vocab_size + self.ingredient_vocab_size), units, mask_zero=True)

    # The RNN layer processes those vectors sequentially.
    self.rnn = tf.keras.layers.Bidirectional(
        merge_mode='sum',
        layer=tf.keras.layers.GRU(units,
                            # Return the sequence and state
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform'))

  def call(self, ingredients, names):

    shape_checker = ShapeChecker()

    # 2. The embedding layer looks up the embedding vector for each token.
    x1 = self.name_embedding(ingredients)
    x2 = self.ingredient_embedding(names)

    #x = self.concat([x1, x2])
    x = x1
    x = self.embedding(x)
    shape_checker(x, 'batch s units')

   
    # 3. The GRU processes the sequence of embeddings.
    x = self.rnn(x)
    shape_checker(x, 'batch s units')
  
    # 4. Returns the new sequence of embeddings.
    return x

  def convert_input(self, ingredients, names):
    ingredients = tf.convert_to_tensor(ingredients)
    names = tf.convert_to_tensor(names)
    
    names = self.name_processor(names)#.to_tensor()
    ingredients = self.ingredient_processor(ingredients)#.to_tensor()
    
    context = self(ingredients, names)
    return context

In [57]:


# Encode the input sequence.
encoder = Encoder(name_vectorize_layer, ingredient_vectorize_layer, UNITS)



In [58]:
for (ex_names) in name_dataset.take(1):
  print(ex_names.numpy()) 
ex_names_tok = name_vectorize_layer(ex_names)
print(ex_names_tok)

b'get the sensation  brownies'
tf.Tensor([ 975   33 1876    7], shape=(4,), dtype=int64)


In [59]:
for (ex_ingredients) in ingredient_dataset.take(1):
  print(ex_ingredients.numpy()) 

ex_ingredients_tok = ingredient_vectorize_layer(ex_ingredients)
print(ex_ingredients_tok)

b'butter, sugar, vanilla, eggs, all-purpose_flour, baking_cocoa, baking_powder, salt, miniature_peppermint_patties'
tf.Tensor([2 3 6 5], shape=(4,), dtype=int64)


In [60]:
for (ex_steps) in step_dataset.take(1):
  print(ex_steps.numpy()) 
ex_steps_tok = step_vectorize_layer(ex_steps)
print(ex_steps_tok)

b'preheat oven to 350 degrees <STEP> grease 13 x 9 baking pan <STEP> in a large bowl , whisk together butter , sugar and vanilla <STEP> add eggs and stir until well combined <STEP> combine dry ingredients and blend well with wet ingredients <STEP> reserve 2 cups of batter and set aside <STEP> spread remaining batter in prepared pan <STEP> arrange peppermint patties in a single layer over batter , about 1 / 2 inch apart <STEP> carefully spread reserved 2 cups of batter on top <STEP> bake for 50- 55 minutes or until brownies begin to pull away from sides of pan <STEP> cool completely on wire rack and cut into squares\n'
tf.Tensor(
[  54   30    7   68   77    2  128  304  217  190   28   20    2    5
    6   52   21   85   49   23   18    3   36    2   15   56    3   22
    8   38  124    2   41   96   59    3  122   38    9  332   59    2
  642   34  136   11   66    3   44   91    2   61   75   66    5  112
   20    2  477  707 1443    5    6  771  135   26   66   55   24   34
  131  2

In [61]:
sample_name = name_dataset.take(1)
sample_ingredient = ingredient_dataset.take(1)
ex_context = encoder(ex_ingredients_tok, ex_names_tok)
#ex_context = encoder.convert_input(ex_ingredient_tok, ex_name_tok)
ex_context

<tf.Tensor: shape=(4, 256, 256), dtype=float32, numpy=
array([[[ 0.02010092,  0.01672063,  0.01243637, ..., -0.04673108,
         -0.01720304,  0.01636258],
        [ 0.02342843,  0.0121458 ,  0.01253262, ..., -0.0494245 ,
         -0.01391483,  0.02257711],
        [ 0.02460716,  0.00909669,  0.01141806, ..., -0.05020532,
         -0.01017253,  0.0267278 ],
        ...,
        [ 0.02240387,  0.00014399,  0.0048982 , ..., -0.04343624,
         -0.00283831,  0.03283363],
        [ 0.02050241, -0.00418426,  0.00397169, ..., -0.03881026,
         -0.00019266,  0.03186321],
        [ 0.0166775 , -0.01035845,  0.0025769 , ..., -0.02918759,
          0.00606153,  0.0303477 ]],

       [[ 0.02010092,  0.01672063,  0.01243637, ..., -0.04673108,
         -0.01720304,  0.01636258],
        [ 0.02342843,  0.0121458 ,  0.01253262, ..., -0.0494245 ,
         -0.01391483,  0.02257711],
        [ 0.02460716,  0.00909669,  0.01141806, ..., -0.05020532,
         -0.01017253,  0.0267278 ],
        ...,

In [62]:
class CrossAttention(tf.keras.layers.Layer):
  def __init__(self, units, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=1, **kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

  def call(self, x, context):

    shape_checker = ShapeChecker()
    shape_checker(x, 'batch t units')
    shape_checker(context, 'batch s units')


    attn_output, attn_scores = self.mha(
        query=x,
        value=context,
        return_attention_scores=True)
    
    shape_checker(x, 'batch t units')
    shape_checker(attn_scores, 'batch heads t s')


    # Cache the attention scores for plotting later.
    attn_scores = tf.reduce_mean(attn_scores, axis=1)
    shape_checker(attn_scores, 'batch t s')
    self.last_attention_weights = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [63]:
attention_layer = CrossAttention(UNITS)

# Attend to the encoded tokens
embed = tf.keras.layers.Embedding(step_vectorize_layer.vocabulary_size(),
                                  output_dim=UNITS, mask_zero=True, name="bliblablub")

ex_tar_embed = embed(ex_steps_tok)
result = attention_layer(ex_tar_embed, ex_context)

print(f'Context sequence, shape (batch, s, units): {ex_context.shape}')
print(f'Target sequence, shape (batch, t, units): {ex_tar_embed.shape}')
print(f'Attention result, shape (batch, t, units): {result.shape}')
print(f'Attention weights, shape (batch, t, s):    {attention_layer.last_attention_weights.shape}')

RuntimeError: Exception encountered when calling layer 'cross_attention_4' (type CrossAttention).

Can't parse shape with different number of dimensions: batch t units (400, 256)

Call arguments received by layer 'cross_attention_4' (type CrossAttention):
  • x=tf.Tensor(shape=(400, 256), dtype=float32)
  • context=tf.Tensor(shape=(4, 256, 256), dtype=float32)

In [ ]:
class Decoder(tf.keras.layers.Layer):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, text_processor, units):
    super(Decoder, self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.word_to_id = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]')
    self.id_to_word = tf.keras.layers.StringLookup(
        vocabulary=text_processor.get_vocabulary(),
        mask_token='', oov_token='[UNK]',
        invert=True)
    self.start_token = self.word_to_id('[START]')
    self.end_token = self.word_to_id('[END]')

    self.units = units


    # 1. The embedding layer converts token IDs to vectors
    self.embedding = tf.keras.layers.Embedding(self.vocab_size,
                                               units, mask_zero=True)

    # 2. The RNN keeps track of what's been generated so far.
    self.rnn = tf.keras.layers.GRU(units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

    # 3. The RNN output will be the query for the attention layer.
    self.attention = CrossAttention(units)

    # 4. This fully connected layer produces the logits for each
    # output token.
    self.output_layer = tf.keras.layers.Dense(self.vocab_size)

In [ ]:
@Decoder.add_method
def call(self,
         context, x,
         state=None,
         return_state=False):  
  # 1. Lookup the embeddings
  x = self.embedding(x)

  # 2. Process the target sequence.
  x, state = self.rnn(x, initial_state=state)
  # 3. Use the RNN output as the query for the attention over the context.
  x = self.attention(x, context)
  self.last_attention_weights = self.attention.last_attention_weights

  # Step 4. Generate logit predictions for the next token.
  logits = self.output_layer(x)

  if return_state:
    return logits, state
  else:
    return logits

In [ ]:
decoder = Decoder(step_vectorize_layer, UNITS)

In [ ]:
logits = decoder(ex_context, ex_steps_tok)

print(f'encoder output shape: (batch, s, units) {ex_context.shape}')
print(f'input target tokens shape: (batch, t) {ex_steps_tok.shape}')
print(f'logits shape shape: (batch, target_vocabulary_size) {logits.shape}')

ValueError: Exception encountered when calling layer 'decoder_3' (type Decoder).

Input 0 of layer "gru_7" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (400, 256)

Call arguments received by layer 'decoder_3' (type Decoder):
  • context=tf.Tensor(shape=(4, 512, 256), dtype=float32)
  • x=tf.Tensor(shape=(400,), dtype=int64)
  • state=None
  • return_state=False

In [ ]:
@Decoder.add_method
def get_initial_state(self, context):
  batch_size = tf.shape(context)[0]
  start_tokens = tf.fill([batch_size, 1], self.start_token)
  done = tf.zeros([batch_size, 1], dtype=tf.bool)
  embedded = self.embedding(start_tokens)
  return start_tokens, done, self.rnn.get_initial_state(embedded)[0]

In [ ]:
@Decoder.add_method
def tokens_to_text(self, tokens):
  words = self.id_to_word(tokens)
  result = tf.strings.reduce_join(words, axis=-1, separator=' ')
  result = tf.strings.regex_replace(result, '^ *\[START\] *', '')
  result = tf.strings.regex_replace(result, ' *\[END\] *$', '')
  return result

In [ ]:
@Decoder.add_method
def get_next_token(self, context, next_token, done, state, temperature = 0.0):
  logits, state = self(
    context, next_token,
    state = state,
    return_state=True) 

  if temperature == 0.0:
    next_token = tf.argmax(logits, axis=-1)
  else:
    logits = logits[:, -1, :]/temperature
    next_token = tf.random.categorical(logits, num_samples=1)

  # If a sequence produces an `end_token`, set it `done`
  done = done | (next_token == self.end_token)
  # Once a sequence is done it only produces 0-padding.
  next_token = tf.where(done, tf.constant(0, dtype=tf.int64), next_token)

  return next_token, done, state

In [ ]:
# Setup the loop variables.
next_token, done, state = decoder.get_initial_state(ex_context)
tokens = []

for n in range(10):
  # Run one step.
  next_token, done, state = decoder.get_next_token(
      ex_context, next_token, done, state, temperature=1.0)
  # Add the token to the output.
  tokens.append(next_token)

# Stack all the tokens together.
tokens = tf.concat(tokens, axis=-1) # (batch, t)

# Convert the tokens back to a a string
result = decoder.tokens_to_text(tokens)
result[:3].numpy()

array([b'browning amazing owls dumping yogurt brim void metric topside muscovado',
       b'camp lrg meats rearrange remelt key 2832 halves reasons fondue',
       b'criss less frog twist lightlygreased nestling floured minuted fallon slighly'],
      dtype=object)

In [ ]:
class Translator(tf.keras.Model):
  @classmethod
  def add_method(cls, fun):
    setattr(cls, fun.__name__, fun)
    return fun

  def __init__(self, units,  name_processor, ingredient_processor, target_text_processor):
    super().__init__()
    # Build the encoder and decoder
    encoder = Encoder( name_processor, ingredient_processor, units)
    decoder = Decoder(target_text_processor, units)

    self.encoder = encoder
    self.decoder = decoder

  def call(self, ingredients, names, steps):
    
    context = self.encoder.convert_input(ingredients, names)
    logits = self.decoder(context, steps)

    #TODO(b/250038731): remove this
    try:
      # Delete the keras mask, so keras doesn't scale the loss+accuracy. 
      del logits._keras_mask
    except AttributeError:
      pass

    return logits

In [ ]:
model = Translator(UNITS, name_vectorize_layer, ingredient_vectorize_layer, step_vectorize_layer)

logits = model(ex_ingredient_tok, ex_name_tok, ex_name_tok)

print(f'Context tokens, shape: (batch, s, units) {ex_name_tok.shape}')
print(f'Target tokens, shape: (batch, t) {ex_tar_in.shape}')
print(f'logits, shape: (batch, t, target_vocabulary_size) {logits.shape}')

NameError: name 'Translator' is not defined